In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, r2_score
from xgboost import XGBRegressor
import mlflow
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# --- 1. CONFIGURER MLFLOW ---
mlflow.set_experiment("Projet_SIRENE_Regression_Monstre")
mlflow.xgboost.autolog()
print("MLflow configuré pour le 'Run Monstrueux'.")

# --- 2. CHARGER LES BONS "MASTER FILES" ---
print("Chargement des 'Master files'...")
df_sirene = pl.read_parquet("../Data/processed/sirene_infos.parquet")
df_bilan = pl.read_parquet("../Data/processed/sirene_bilan.parquet")
print("Fichiers 'infos' et 'bilan' chargés.")

# --- 3. DÉFINIR LES CODES "DIAMANT" ---
RAW_CODES = [
    'HN_RésultatNet', 'FA_ChiffreAffairesVentes', 'FB_AchatsMarchandises',
    'CJCK_TotalActifBrut', 'DL_DettesCourtTerme', 'DM_DettesLongTerme',
    'DA_TresorerieActive', 'FJ_ResultatFinancier', 'FR_ResultatExceptionnel',
    'DF_CapitauxPropres', 'EG_ImpotsTaxes'
]
RATIO_CODES = [
    "ratio_rentabilite_nette", "ratio_endettement", "ratio_marge_brute", 
    "ratio_capitaux_propres", "ratio_tresorerie",
    "ratio_resultat_financier", "ratio_resultat_exceptionnel"
]
FINANCIAL_FEATURES = RAW_CODES + RATIO_CODES
CATEGORICAL_FEATURES = ["categorieJuridiqueUniteLegale", "departement"]
TARGET = "TARGET_rentabilite_N"

# --- 4. CRÉATION DU DATASET (LE "SELF-JOIN" TEMPOREL) ---
print("Création du dataset temporel (N, N-1, N-2)...")

# A. Données N (2019) - LA TARGET
df_N = df_bilan.filter(pl.col("AnneeClotureExercice") == 2019).select(
    "siren", pl.col("ratio_rentabilite_nette").alias(TARGET)
)

# B. Données N-1 (2018) - Features "État"
df_N_moins_1 = df_bilan.filter(pl.col("AnneeClotureExercice") == 2018).select(
    "siren", *[pl.col(c).alias(f"{c}_N1") for c in FINANCIAL_FEATURES]
)

# C. Données N-2 (2017) - Features "Historique"
df_N_moins_2 = df_bilan.filter(pl.col("AnneeClotureExercice") == 2017).select(
    "siren", *[pl.col(c).alias(f"{c}_N2") for c in FINANCIAL_FEATURES]
)

# --- 5. LE FEATURE ENGINEERING "MONSTRUEUX" ---
print("Création des features de 'Vélocité' et 'Accélération'...")
# On joint N-1 et N-2
df_features = df_N_moins_1.join(df_N_moins_2, on="siren", how="left").fill_null(0)

# On crée les features de "Vélocité" (Variation N-1 vs N-2)
for c in FINANCIAL_FEATURES:
    df_features = df_features.with_columns(
        (pl.col(f"{c}_N1") - pl.col(f"{c}_N2")).alias(f"var_{c}_N1_N2")
    )

# On joint avec les features "Démo" (le châssis !)
df_features = df_features.join(
    df_sirene.select("siren", *CATEGORICAL_FEATURES),
    on="siren",
    how="left"
)

# --- 6. JOINTURE FINALE (Features + Target) ---
df_ml = df_features.join(df_N, on="siren", how="inner")
print(f"Dataset de Régression 'Monstre' créé. Shape: {df_ml.shape}")

# --- 7. DÉFINITION FINALE DES FEATURES (X) ET TARGET (Y) ---
# On prend TOUT : État N-1, État N-2, Variations, et Démo
NUMERIC_FEATURES_FINAL = [c for c in df_ml.columns if c not in ["siren", TARGET] + CATEGORICAL_FEATURES]
print(f"Total features: {len(CATEGORICAL_FEATURES)} cat + {len(NUMERIC_FEATURES_FINAL)} num.")

# --- 8. NETTOYAGE DES OUTLIERS (Clipping) ---
print("Clipping des outliers...")
LOWER_BOUND, UPPER_BOUND = -5.0, 5.0
clip_cols = [c for c in NUMERIC_FEATURES_FINAL if "ratio" in c or "variation" in c]
df_ml = df_ml.with_columns(
    pl.col(clip_cols).clip(lower_bound=LOWER_BOUND, upper_bound=UPPER_BOUND),
    pl.col(TARGET).clip(lower_bound=LOWER_BOUND, upper_bound=UPPER_BOUND)
).fill_null(0)

# Conversion en Pandas
X = df_ml.select(NUMERIC_FEATURES_FINAL + CATEGORICAL_FEATURES).to_pandas()
y = df_ml.select(TARGET).to_pandas().squeeze()

# --- 9. PRÉPARATION (Le Preprocessor "Monstre") ---
print("Preprocessing avec RobustScaler (Num) + OHE (Cat)...")
numerical_transformer = RobustScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, NUMERIC_FEATURES_FINAL),
        ("cat", categorical_transformer, CATEGORICAL_FEATURES)
    ],
    remainder="passthrough"
)

# --- 10. CRÉATION DE LA PIPELINE ET TUNING ---
print("Création de la pipeline (Preprocessor + XGB Regressor)...")
pipeline_preprocessor = Pipeline(steps=[('preprocessor', preprocessor)])
print("Preprocessing terminé. Lancement du tuning (peut prendre 10-20 minutes)...")

# Grille de tuning plus large
param_grid = {
    'n_estimators': [100, 250, 400],
    'max_depth': [5, 7, 10],
    'learning_rate': [0.1, 0.05, 0.01],
    'subsample': [0.7, 1.0] # Ajout de subsample pour la robustesse
}
# Total d'expériences : 3 * 3 * 3 * 2 = 54 (x 5 folds = 270 entraînements)
# C'EST LENT. On va utiliser RandomizedSearchCV pour en tester 20.

from sklearn.model_selection import RandomizedSearchCV

xgb_reg = XGBRegressor(objective='reg:squarederror', eval_metric='rmse', random_state=42)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# On utilise RandomizedSearchCV pour aller plus vite
random_search = RandomizedSearchCV(
    estimator=xgb_reg,
    param_distributions=param_grid,
    n_iter=20, # On ne teste que 20 combinaisons au hasard
    cv=kfold,
    scoring='r2',
    verbose=2,
    n_jobs=-1,
    random_state=42
)

X_processed = pipeline_preprocessor.fit_transform(X)

with mlflow.start_run() as run:
    random_search.fit(X_processed, y)
    mlflow.log_param("model_type", "Model_H_Full_Velocity_Tuned")
    mlflow.log_metric("best_r2_score", random_search.best_score_)

# --- 11. RÉSULTATS DU TUNING "MONSTRUEUX" ---
print("---")
print("--- RÉSULTATS DU TUNING 'MONSTRUEUX' (RANDOMIZEDSEARCHCV) ---")
print(f"Meilleur Score R² trouvé : {random_search.best_score_:.4f}")
print("Meilleurs Hyperparamètres :")
print(random_search.best_params_)
print("---")
print(f"Score précédent (Modèle F, non-tuné): 0.3091")
print(f"Score actuel (Modèle H, tuné): {random_search.best_score_:.4f}")
print("---")
print("Toutes les expériences sont loggées dans 'mlruns'.")
print("Lance 'mlflow ui' dans ton terminal pour voir le dashboard.")

/Users/ugo/Documents/Cours/Master/Cours/M1/Supervised Learning/Final Project/.venv/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/16 17:48:22 INFO mlflow.tracking.fluent: Experiment with name 'Projet_SIRENE_Regression_Monstre' does not exist. Creating a new experiment.


MLflow configuré pour le 'Run Monstrueux'.
Chargement des 'Master files'...
Fichiers 'infos' et 'bilan' chargés.
Création du dataset temporel (N, N-1, N-2)...
Création des features de 'Vélocité' et 'Accélération'...
Dataset de Régression 'Monstre' créé. Shape: (415027, 58)
Total features: 2 cat + 54 num.
Clipping des outliers...
Preprocessing avec RobustScaler (Num) + OHE (Cat)...
Création de la pipeline (Preprocessor + XGB Regressor)...
Preprocessing terminé. Lancement du tuning (peut prendre 10-20 minutes)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END learning_rate=0.05, max_depth=5, n_estimators=100, subsample=1.0; total time=  20.2s
[CV] END learning_rate=0.05, max_depth=5, n_estimators=100, subsample=1.0; total time=  20.5s
[CV] END learning_rate=0.05, max_depth=5, n_estimators=100, subsample=1.0; total time=  20.7s
[CV] END learning_rate=0.05, max_depth=5, n_estimators=100, subsample=1.0; total time=  21.0s
[CV] END learning_rate=0.05, max_depth=5, n_e

2025/11/16 17:53:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


---
--- RÉSULTATS DU TUNING 'MONSTRUEUX' (RANDOMIZEDSEARCHCV) ---
Meilleur Score R² trouvé : 0.3294
Meilleurs Hyperparamètres :
{'subsample': 0.7, 'n_estimators': 400, 'max_depth': 10, 'learning_rate': 0.01}
---
Score précédent (Modèle F, non-tuné): 0.3091
Score actuel (Modèle H, tuné): 0.3294
---
Toutes les expériences sont loggées dans 'mlruns'.
Lance 'mlflow ui' dans ton terminal pour voir le dashboard.
